In [1]:
import cv2
import numpy as np
import ipywidgets as widgets
from IPython.display import display
from jetracer.nvidia_racecar import NvidiaRacecar
from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg

# Initialize the car
car = NvidiaRacecar()
car.steering_gain = 1.0

def find_yellow_duck_and_steering_angle(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define the range of yellow color in HSV
    lower_yellow = np.array([20, 100, 100])
    upper_yellow = np.array([30, 255, 255])

    # Create a mask for yellow color
    mask = cv2.inRange(hsv, lower_yellow, upper_yellow)

    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        largest_contour = max(contours, key=cv2.contourArea)

        # Get the bounding box of the largest contour
        x, y, w, h = cv2.boundingRect(largest_contour)

        # Calculate the center of the bounding box
        center_x = x + w // 2
        image_center_x = image.shape[1] // 2

        # Calculate the steering angle
        steering_angle = (center_x - image_center_x) / image_center_x

        # Normalize the steering angle to be within [-1, 1]
        steering_angle = np.clip(steering_angle, -1, 1)

        # Calculate the area of the bounding box (used for throttle control)
        bounding_box_area = w * h

        # Draw the bounding box and center point on the image
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.circle(image, (center_x, y + h // 2), 5, (0, 255, 0), -1)

        return image, steering_angle, bounding_box_area
    else:
        return image, None, None

def control_car(steering_angle, bounding_box_area, target_area=5000):
    # Steering control
    car.steering = steering_angle
    
    # Throttle control: Move forward if the duck is far, stop if close
    if bounding_box_area is not None:
        if bounding_box_area < target_area:
            car.throttle = 0.3  # Move forward
        else:
            car.throttle = 0  # Stop
    else:
        car.throttle = 0  # Stop if no duck is detected


# Initialize the CSICamera
camera = CSICamera(width=224, height=224, capture_fps=30)

# Create an IPython image widget
image_widget = widgets.Image(format='jpeg', width=224, height=224)
steering_slider = widgets.FloatSlider(description='steering', min=-1.0, max=1.0, orientation='vertical')

display(widgets.HBox([image_widget, steering_slider]))
    

WARNNIG: Jetson.GPIO library has not been verified with this carrier board,


In [2]:
while True:
        # Capture a frame from the camera
        frame = camera.read()

        # Process the frame to find the duck and calculate steering angle
        processed_frame, steering_angle, bounding_box_area = find_yellow_duck_and_steering_angle(frame)
        
        if steering_angle is not None:
            control_car(steering_angle, bounding_box_area)
            steering_slider.value = steering_angle
        else:
            car.throttle = 0  # Stop if no duck is detected

        # Update the image widget
        image_widget.value = bgr8_to_jpeg(processed_frame)


KeyboardInterrupt: 

In [ ]:
car.throttle = 0.0